# GoogleHydrology Finetuning Tutorial Notebook: Exploring Base Models and Targeted Fine-Tuning

Welcome to this interactive tutorial designed to help you understand the concepts of base hydrological models and the powerful technique of fine-tuning, using the `googlehydrology` framework. This notebook is structured as a student exercise, aiming to provide a hands-on experience in evaluating and improving predictive models.

## What You Will Learn:

By working through this notebook, you will gain insights into:

1.  **Understanding Base Models:** How a general-purpose hydrological model, trained on a diverse set of basins, serves as a foundation for further specialization.
2.  **The Concept of Fine-Tuning:** The process of adapting a pre-trained model to improve its performance on a specific target or region that might be 'out-of-distribution' compared to the base model's training data.
3.  **Running `googlehydrology` Models:** Practical experience in generating and understanding the command-line arguments for training (`train`), fine-tuning (`finetune`), and performing inference (`infer`) with the `googlehydrology` framework. You will learn how to prepare config files and execute these operations, typically in a terminal environment.
4.  **Model Performance Analysis:** How to quantitatively evaluate model performance using various metrics (e.g., KGE, NSE) and qualitatively assess predictions through hydrograph comparisons.
5.  **Impact of Static Attributes:** The critical role of static basin attributes (like basin area) in model performance and how targeted fine-tuning of specific model layers (e.g., `static_attributes_fc`) can address discrepancies when a target basin's characteristics differ significantly from the base model's training distribution.

For more detailed insights into fine-tuning for river modeling, refer to the paper: Ryd, Emil, and Grey Nearing. "Fine Flood Forecasts: Incorporating local data into global models through fine-tuning." arXiv preprint arXiv:2504.12559 (2025). [https://arxiv.org/abs/2504.12559](https://arxiv.org/abs/2504.12559)

## Notebook Workflow:

This notebook follows a structured approach to guide you through the comparison of a **base model** and a corresponding **fine-tuned model**. The key steps include:

1.  **Configuration:** Setting up local paths for model runs, shapefiles, and attribute data.
2.  **Base Model Exploration:** Selecting a base model, visualizing its training and testing basins, generating its run commands, calculating performance metrics, and visualizing score distributions.
3.  **Fine-Tuning Process:** Choosing a specific target basin for fine-tuning, understanding how to configure the fine-tuning run, and generating the necessary fine-tuning and inference commands.
4.  **Fine-Tuned Model Analysis:** Selecting the fine-tuned model and loading its performance metrics.
5.  **Comparative Analysis:** Directly comparing the base and fine-tuned models' performance on the target basin using both metric-versus-lead-time plots and detailed hydrograph comparisons.
6.  **Basin Context:** Examining the basin area distribution of the training data relative to the fine-tuning target basin to understand the rationale behind targeted fine-tuning strategies, particularly concerning the `static_attributes_fc` layer.

By the end of this tutorial, you should have a solid understanding of how to leverage pre-trained hydrological models and adapt them for specific regional challenges, ultimately contributing to more accurate flood forecasting.

This notebook is designed as an educational exercise rather than a performance benchmark. To ensure the code runs quickly in a standard environment (like Google Colab), the experiment is restricted to a "toy" dataset of only 5 training basins. Because State-of-the-Art (SOTA) global models typically require data from hundreds or thousands of basins to learn universal hydrologic behaviors and relationships, this 5-basin model will **not** yield state-of-the-art results. Specifically, a model trained on such a small sample size lacks the "experience" to understand basins in different climates or terrains. You will observe that performance metrics (NSE/KGE) on the 3 "ungauged" basins (basins not seen during training) are significantly lower than the training set. This is expected behavior from a model trained on a small (5-basin) dataset.

## Imports

In [ ]:
# # --- Install Dependencies ---
# # These lines ensure that all required third-party libraries are installed.
# # These are not needed if you use the supplied `googlehydrology` conda
# # environment in `~/flood-forecasting/environments/conda.yml`

# # Data visualization and interactive widgets
# !pip install -q matplotlib seaborn ipywidgets

# # Scientific computing and data structures
# !pip install -q numpy pandas xarray netCDF4

# # Google Cloud Storage (for MultiMet data access)
# !pip install -q google-cloud-storage gcsfs

In [ ]:
# Standard Library Imports
import os
import glob
import yaml
from typing import Dict, List, Optional, Set

# Scientific Computing & Data Analysis
import numpy as np
import pandas as pd
import xarray as xr

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive Widgets for Notebook Environments
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive

# Local Tutorial Module
import backend

In [ ]:
%matplotlib inline

## User-Defined Local Paths


In [ ]:
# Define the path to the shapefile containing basin geometries.
# This shapefile is used for plotting maps of basin locations.
# Users may need to update this path if their data is stored elsewhere.
SHAPEFILE_PATH = '~/flood-forecasting/tutorial/Caravan-nc/shapefiles/camels/camels_basin_shapes.shx'

# Path to the attributes file that contains additional basin information, such as basin area.
# This file is used for visualizing the distribution of basin areas.
ATTRIBUTES_FILE_PATH = '~/flood-forecasting/tutorial/Caravan-nc/attributes/camels/attributes_other_camels.csv'

# Path to a base directory containing one or more model run directories.
# This directory should house all your trained models.
# The interactive selection widgets in subsequent cells will scan this directory to allow you to choose which model run to evaluate.
MODEL_RUN_DIR = '/home/gsnearing/flood-forecasting/tutorial/model-runs'

# The directory where .yml model configuration files are located.
# These files define parameters for model runs, including data paths and training settings.
CONFIG_DIR = '/home/gsnearing/flood-forecasting/tutorial/configs'

# --- OPTIONAL: Full Training Pipeline Trigger ---
# By default, this tutorial uses pre-trained model outputs to save time.
# Set this flag to 'True' ONLY if you want to perform a fresh training run
# and generate new predictions from scratch.
RUN_NEW_MODELS = False

# A boolean flag to control whether performance statistics (metrics) are recalculated or loaded from pre-existing files.
# Set this to 'True' for the initial run or if model outputs have changed and you need fresh calculations.
# Set to 'False' if metrics have already been computed and saved, to save time during subsequent runs.
# This is particularly useful for live demos or quick comparisons where recalculation is not necessary.
CALCULATE_STATISTICS = False

## Interactive Functions

In [ ]:
def select_model(model_selection):
    """
    Sets the global variables for the selected base model run directory and name.

    This function is designed to be used with an `ipywidgets.Dropdown` to allow
    interactive selection of a model. It updates two global variables:
    - `model_run_dir`: The absolute path to the directory of the selected model run.
    - `model_name`: A user-friendly display name for the selected model.

    Args:
        model_selection (str): The key (display name) of the selected model run
                                from the available options in the dropdown.
    """
    global model_run_dir, model_name

    # Get the absolute path for the selected run directory from the pre-populated dictionary
    model_run_dir = available_run_dirs.get(model_selection)
    # Create a display name for the selected model, including its selection identifier
    model_name = f'Model ({model_selection})'

    print("\nConfiguration set:")
    print(f"  Model: {model_name} ({model_run_dir})")

## Base Model

### Training & Inference Pipelines

#### Run Commands

In [ ]:
# --- Base Model: Generate Run Commands ---

# This cell generates the commands needed to train and run inference for the base model.
# These commands are typically executed outside of this notebook in a terminal or a separate process
# that has access to the `flood-forecasting/googlehydrology` project and its
# required Python environment (e.g., a Conda environment).

# Get the experiment name. We are assuming '5-basin-example' as the base experiment
# because the base_config may not be loaded yet at this stage.
training_config_file = 'train-config.yml'
base_config_file = f"{CONFIG_DIR}/{training_config_file}"

# Construct the base model training command.
# This command initiates the training process for the base model, using its specific configuration file.
base_train_command = f"run train --config-file={base_config_file}"

# Construct the base model inference command.
# After a base model has been trained, this command is used to run predictions
# (inference) using the trained model. It specifies the `--run-dir`
# which points to the output directory of the base model training run.
# NOTE: At this stage, the actual run directory might not exist yet if you are training a new model.
# You will need to replace '<YOUR_BASE_MODEL_RUN_DIR>' with the actual directory name created by STEP 1.
base_infer_command = f"run infer --run-dir={MODEL_RUN_DIR}<YOUR_BASE_MODEL_RUN_DIR>"

# Print the run instructions to the screen
print("--------------------------------------------------------------------------------------------------")
print("                          BASE MODEL TRAINING AND INFERENCE INSTRUCTIONS                          ")
print("--------------------------------------------------------------------------------------------------")
print("These commands need to be executed in your terminal or a separate process.")
print("Ensure you are in an environment with the necessary Python libraries (e.g., the `googlehydrology` Conda environment).")
print("")
print("IMPORTANT: Before running these commands, please examine the configuration file:")
print(f"  Config File Location: {base_config_file}")
print("  This file defines crucial parameters for your model run, including data paths, model architecture, and training settings.")
print("  Understanding its contents will help you customize and debug your experiments.")
print("")
print("STEP 1: Run the base model training command FIRST (if you need to train a new base model):")
print(f"    {base_train_command}")
print("")
print("STEP 2: After the base model training is complete, run the inference command:")
print("    (IMPORTANT: Replace '<YOUR_BASE_MODEL_RUN_DIR>' with the actual run directory created by STEP 1, e.g., 'model-runs/5-basin-example_YYYYMMDD_HHMMSS')")
print(f"    {base_infer_command}")
print("")
print("--------------------------------------------------------------------------------------------------")

#### Run Training & Inference

In [ ]:
if RUN_NEW_MODELS:
    # 1. Execute Base Model Training:
    # This calls the 'run train' command with your specified YAML config.
    # WARNING: Full training can take a significant amount of time (minutes to hours)
    # depending on your hardware (CPU/GPU) and the number of basins.
    print(f"Executing training: {base_train_command}")
    os.system(base_train_command)

    # Load the base config file to get the actual experiment name
    with open(base_config_file, 'r') as f:
        base_config_data = yaml.safe_load(f)
    actual_experiment_name = base_config_data.get('experiment_name', 'default_experiment')
    print(f"Experiment name from config file: {actual_experiment_name}")

    # Get all potential run directories for the base experiment
    search_pattern = os.path.join(MODEL_RUN_DIR, f'{actual_experiment_name}_*_*/')
    all_run_dirs = glob.glob(search_pattern)

    if not all_run_dirs:
        print(f"Error: No run directories found matching pattern {search_pattern}")
    else:
        # Sort them by modification time to get the most recent one
        # Or, parse the timestamp from the directory name if it's consistently formatted
        # For now, let's assume the default `googlehydrology` naming `experiment_YYYYMMDD_HHMMSS` makes lexicographical sort work.
        all_run_dirs.sort()
        base_model_actual_run_dir = all_run_dirs[-1].rstrip('/') # Get the last (most recent) and remove trailing slash
        print(f"Found most recent base model run directory: {base_model_actual_run_dir}")

        # 2. Execute Inference:
        # After training completes, this command generates the actual streamflow
        # predictions (.nc or .zarr files) used for the hydrographs later in this notebook.
        infer_command = f"run infer --run-dir={base_model_actual_run_dir}"
        print(f"Executing inference: {infer_command}")
        os.system(infer_command)
else:
    print("Skipping re-training. You will select the base model in the next step.")

### Choose Base Model

This section allows you to select a **base model** from your local `MODEL_RUN_DIR`. The notebook automatically scans this directory (including subdirectories) to find all available model run folders. Each folder represents a unique experiment or training run.

An interactive dropdown widget is then populated with the names of these detected runs. Selecting an option from this dropdown will set the `model_run_dir` and `model_name` global variables, which are used throughout the rest of the notebook for loading data and calculating metrics for the chosen base model.


In [ ]:
# Find available run directories using the path set in the cell above
available_run_dirs = backend.find_model_run_dirs(MODEL_RUN_DIR)

# Extract the name of the most recent model run for default selection
if 'base_model_actual_run_dir' in globals():
    latest_run_name = os.path.basename(base_model_actual_run_dir)
else:
    latest_run_name = None

# Create a dropdown widget for selecting the base model
run_dir_options = sorted(available_run_dirs.keys())
base_dropdown = widgets.Dropdown(
    options=run_dir_options,
    description='Select Base Model:',
    disabled=False,
    style = {'description_width': 'initial'},
    value=latest_run_name if latest_run_name in run_dir_options else run_dir_options[0]
)

# Create an interactive widget linking the dropdown to the selection function
interactive_selection = interactive(
    select_model,
    model_selection=base_dropdown,
)

# Display the interactive widget
display(interactive_selection)

### Visualize Train and Test Basins

In [ ]:
# Load train and test basin lists
# `backend.load_model_config_and_basins` reads the configuration file of the selected base model
# (from the `model_run_dir`) to extract which basins were used for training and testing.
# It returns the configuration dictionary, and sets of training and testing basin IDs.
base_config, train_basin_ids, test_basin_ids = backend.load_model_config_and_basins(model_run_dir)

# Plot train and test basins
# `backend.plot_train_test_shapefile` visualizes the geographical locations of the
# training and testing basins on a map. It uses the `SHAPEFILE_PATH` to draw the basin
# boundaries, highlights `train_basin_ids` and `test_basin_ids` with different colors,
# and uses `model_name` for the plot title.
backend.plot_train_test_shapefile(
    shapefile_path=SHAPEFILE_PATH,
    train_basin_ids=train_basin_ids,
    test_basin_ids=test_basin_ids,
    model_name=model_name
)

### Calculate Metrics

In [ ]:
# Load simulation data and calculate/load metrics for the base model.
# `backend.load_data_and_metrics` handles reading the raw model outputs
# and, if specified, computes performance metrics across different lead times.
# It requires the `model_run_dir` to locate the data and `test_basin_ids` to filter for relevant basins.
base_model_data, base_model_metrics = backend.load_data_and_metrics(
    model_run_dir=model_run_dir,
    test_basin_ids=test_basin_ids,
    # The 'calculate_statistics' parameter controls whether metrics are re-calculated
    # or loaded from a pre-saved file. Setting it to `True` (as done here temporarily)
    # forces recalculation, which is useful after a new model run or if previous
    # calculations are outdated. If set to `False` (referencing the global
    # CALCULATE_STATISTICS), it will load existing metrics if available, saving time.
    calculate_statistics=CALCULATE_STATISTICS,
    model_name=model_name
)

# `base_model_data` will contain the raw simulation outputs (e.g., streamflow_sim, streamflow_obs)
# typically as an xarray Dataset, allowing for detailed hydrograph analysis.
# `base_model_metrics` will be a pandas DataFrame containing calculated performance scores
# (e.g., KGE, NSE) for each basin and lead time, used for quantitative comparisons.
display(base_model_metrics.head())

### Visualize Score Distributions

In [ ]:
# Get the list of available metrics from the columns of the base model metrics DataFrame.
# This ensures the dropdown only presents metrics that have actually been calculated and are present.
available_metrics = list(base_model_metrics.columns)

# Create a dropdown widget for selecting a metric to plot at lead time 0.
# The default value is 'KGE' if available, otherwise it defaults to the first metric in the list.
metric_widget = widgets.Dropdown(
    options=available_metrics,
    value='KGE' if 'KGE' in available_metrics else available_metrics[0],
    description='Select Metric for Lead Time 0 Plot:',
    disabled=False,
    style = {'description_width': 'initial'}
)

# Create an interactive widget that links the 'metric_widget' dropdown to the 'plot_lead_time_zero_scores' function.
# `widgets.fixed` is used to pass static arguments (like dataframes and basin IDs) that do not change
# when the dropdown value is altered, making the plot update dynamically only based on the selected metric.
interactive_plot = interactive(
    backend.plot_lead_time_zero_scores,
    metrics_df=widgets.fixed(base_model_metrics),
    train_basin_ids=widgets.fixed(train_basin_ids),
    test_basin_ids=widgets.fixed(test_basin_ids),
    metric_name=metric_widget,
    model_name=widgets.fixed(model_name if 'model_name' in globals() else 'Base Model')
)

# Display the interactive widget, which includes the dropdown and the output plot.
display(interactive_plot)

## Fine Tuning

### Choose Basin

In [ ]:
# --- Fine-Tuning: Choose Basin for Fine-Tuning ---

# This cell is dedicated to interactively selecting and visualizing the specific
# basin that will be targeted for fine-tuning. The goal is to improve the
# model's performance specifically for this chosen basin.

# Initial hardcoded value for FINE_TUNING_BASIN. This will be the default
# selection in the dropdown and will be used if no interactive selection occurs.
# Example for CAMELS dataset: Basin with a low KGE score.
# You can change this value to any valid basin ID from your dataset.
DEFAULT_FINE_TUNING_BASIN_ID = 'camels_13235000' # Example for small model run in live demo.

# Get the list of test basin IDs from the base model configuration.
# These are the basins on which the base model was evaluated and from which
# we can select a target for fine-tuning.
# `test_basin_ids` is expected to be available globally from previous cells
# (e.g., after `backend.load_model_config_and_basins` in the 'Base Model' section).
fine_tuning_basin_options = sorted(list(test_basin_ids))

# Create a dropdown widget for selecting the fine-tuning basin.
# The options are derived from the `test_basin_ids` of the base model.
# The default value is set to `DEFAULT_FINE_TUNING_BASIN_ID` if it exists
# in the options, otherwise it defaults to the first available basin.
basin_selector_dropdown = widgets.Dropdown(
    options=fine_tuning_basin_options,
    value=DEFAULT_FINE_TUNING_BASIN_ID if DEFAULT_FINE_TUNING_BASIN_ID in fine_tuning_basin_options else fine_tuning_basin_options[0],
    description='Select Fine-Tuning Basin:',
    disabled=False,
    style = {'description_width': 'initial'}
)

# Define a function to update the global FINE_TUNING_BASIN and plot
# when a new basin is selected from the dropdown.
def select_fine_tuning_basin(basin_id_selection):
    global FINE_TUNING_BASIN
    FINE_TUNING_BASIN = basin_id_selection
    print(f"\nSelected Fine-Tuning Basin: {FINE_TUNING_BASIN}")

    # Plot the selected fine-tuning basin using the shapefile function.
    # This visualization helps confirm that the correct basin has been selected
    # and allows for a geographical context of the fine-tuning target.
    # `backend.plot_train_test_shapefile` is a utility function that draws basin boundaries
    # on a map, highlighting specific basins.
    backend.plot_train_test_shapefile(
        shapefile_path=SHAPEFILE_PATH,         # Path to the shapefile containing basin geometries.
        train_basin_ids=set(),                # An empty set because we are not highlighting training basins here.
        test_basin_ids={FINE_TUNING_BASIN},   # A set containing only the chosen fine-tuning basin ID to highlight it.
        model_name=f"Fine-Tuning Basin: {FINE_TUNING_BASIN}" # Updates the plot title to indicate the fine-tuning basin.
    )

# Create an interactive widget linking the dropdown to the selection function.
# This displays the dropdown and automatically calls `select_fine_tuning_basin`
# whenever the dropdown value changes.
interactive_basin_selection = interactive(
    select_fine_tuning_basin,
    basin_id_selection=basin_selector_dropdown,
)

# Display the interactive widget to the user.
display(interactive_basin_selection)

# Initialize FINE_TUNING_BASIN with the default or first value initially.
# This ensures FINE_TUNING_BASIN is set even before the user interacts with the dropdown.
if 'FINE_TUNING_BASIN' not in globals():
    FINE_TUNING_BASIN = basin_selector_dropdown.value
    print(f"Initial Fine-Tuning Basin: {FINE_TUNING_BASIN}")

In [ ]:
finetune_basin_kge_scores = base_model_metrics.loc[f'{FINE_TUNING_BASIN}', 'KGE']

# Create a simple bar chart
plt.figure(figsize=(10, 6))
plt.bar(finetune_basin_kge_scores.index, finetune_basin_kge_scores.values)

plt.title(f"Base Model KGE vs. Lead Time for Basin {FINE_TUNING_BASIN}")
plt.xlabel("Lead Time (days)")
plt.ylabel("KGE Score")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(finetune_basin_kge_scores.index)
plt.show()

### Finetuning & Inference Pipelines

#### Create Finetuning Config File

In [ ]:
# --- Configuration File Paths ---
finetune_config_template_path = f'{CONFIG_DIR}/finetune-config.yml'
finetune_basin_config_file = f'{CONFIG_DIR}/finetune-config-{FINE_TUNING_BASIN}.yml'

# --- Execute Config Generation ---

# 1. Create the basin list file
backend.create_basin_list_file(FINE_TUNING_BASIN, output_dir='basin-lists')

# 2. Generate the config file
# Construct the full path to the base model run directory
base_model_run_dir_path = os.path.join(MODEL_RUN_DIR, os.path.basename(model_run_dir))

backend.generate_basin_finetune_config(
    template_path=finetune_config_template_path,
    basin_id=FINE_TUNING_BASIN,
    base_model_dir=base_model_run_dir_path,
    output_path=finetune_basin_config_file
)

#### Run Command

In [ ]:
# --- Fine-Tuning: Generate Run Command ---

# This cell generates the commands needed to run the fine-tuning experiment
# and subsequent inference for the selected basin.

# Assumes FINE_TUNING_BASIN and finetune_basin_config_file are available globally.

# Construct the fine-tuning training command.
# This points to the basin-specific config file we just generated.
finetune_train_command = f"run finetune --config-file={finetune_basin_config_file}"

# Construct the fine-tuning inference command.
# The run directory will be inside the base model directory, starting with the experiment name.
# Note: The actual directory will have a timestamp suffix.
finetune_experiment_name = f"finetune-{FINE_TUNING_BASIN}"
finetune_infer_command_display = f"run infer --run-dir={model_run_dir}/{finetune_experiment_name}_YYYYMMDD_HHMMSS"


# Print the run instructions to the screen
print("--------------------------------------------------------------------------------------------------")
print("                       FINE-TUNING AND INFERENCE RUN INSTRUCTIONS                           ")
print("--------------------------------------------------------------------------------------------------")
print("These commands need to be executed in your terminal or a separate process.")
print("Ensure you are in an environment with the necessary Python libraries (e.g., the `googlehydrology` Conda environment).")
print("")
print("IMPORTANT: The configuration file has been automatically generated for your selected basin:")
print(f"  Config File Location: {finetune_basin_config_file}")
print("  This file contains the specific settings for fine-tuning on basin " + str(FINE_TUNING_BASIN))
print("  It has been pre-configured with the correct base model paths and basin list.")
print("")
print("STEP 1: Run the fine-tuning training command:")
print(f"    {finetune_train_command}")
print("")
print("STEP 2: After training is complete, find the created run directory and run inference.")
print(f"    The output directory will be created inside: {model_run_dir}")
print(f"    It will be named like: {finetune_experiment_name}_YYYYMMDD_HHMMSS")
print("")
print("    Run the inference command using that directory:")
print(f"    {finetune_infer_command_display}")
print("")
print("--------------------------------------------------------------------------------------------------")

#### Run Finetuning & Inference

In [ ]:
if RUN_NEW_MODELS:
    # 1. Execute Targeted Fine-Tuning:
    # This calls 'run finetune' using the basin-specific YAML config.
    # It freezes most of the model (LSTMs) and only updates the 'static_attributes_fc'
    # and 'head' layers to better represent the unique characteristics of this basin.
    print(f"Executing fine-tuning: {finetune_train_command}")
    os.system(finetune_train_command)

    # Load the fine-tune config file to get the actual experiment name
    # We use the dynamically generated config file path from the previous steps
    with open(finetune_basin_config_file, 'r') as f:
        finetune_config_data = yaml.safe_load(f)
    actual_finetune_experiment_name = finetune_config_data.get('experiment_name', 'default_finetune_experiment')
    print(f"Finetune experiment name from config file: {actual_finetune_experiment_name}")

    # Get all potential run directories for the fine-tune experiment
    # These are expected to be within the base model's run directory structure.
    finetune_search_pattern = os.path.join(model_run_dir, f'{actual_finetune_experiment_name}_*_*/')
    all_finetune_run_dirs = glob.glob(finetune_search_pattern)

    if not all_finetune_run_dirs:
        print(f"Error: No fine-tune run directories found matching pattern {finetune_search_pattern}")
    else:
        all_finetune_run_dirs.sort()
        finetune_model_actual_run_dir = all_finetune_run_dirs[-1].rstrip('/')
        print(f"Found most recent fine-tune model run directory: {finetune_model_actual_run_dir}")

        # 2. Execute Inference on Fine-Tuned Model:
        # Once fine-tuning is complete, this command generates new predictions
        # using the updated weights.
        finetune_infer_command_actual = f"run infer --run-dir={finetune_model_actual_run_dir}"
        print(f"Executing fine-tuned inference: {finetune_infer_command_actual}")
        os.system(finetune_infer_command_actual)
else:
    print("Skipping fresh fine-tuning. Loading existing fine-tuned artifacts from the run directory.")

### Select Fine Tuning Model Run

In [ ]:
# --- Fine-Tuning: Select Fine-Tuning Model Run ---

# Define the expected experiment name based on the selected basin
finetune_experiment_name = f"finetune-{FINE_TUNING_BASIN}"

# Try to find the run directory dynamically
# This handles both cases:
# 1. A new run was just created (with a timestamp).
# 2. We are loading an existing run from disk.

# Search for directories matching the experiment name pattern inside the base model directory
search_path = os.path.join(model_run_dir, f"{finetune_experiment_name}*")
found_dirs = glob.glob(search_path)

if found_dirs:
    # Sort to find the most recent one (lexicographical sort works for standard timestamps)
    found_dirs.sort()
    finetune_run_dir = found_dirs[-1]
else:
    # Fallback if no directory is found (e.g., if the model hasn't been run yet)
    finetune_run_dir = os.path.join(model_run_dir, finetune_experiment_name)
    print(f"Warning: No run directory found matching '{finetune_experiment_name}*'. Defaulting to: {finetune_run_dir}")

# Update the global variables to point to the fine-tuned model.
# This ensures that the next cell (metrics calculation) loads data from this new directory.
model_name = f'Fine-Tuned Model ({FINE_TUNING_BASIN})'

print(f"Selected Fine-Tuned Model: {model_name}")
print(f"Run Directory: {finetune_run_dir}")

### Calculate Metrics

In [ ]:
# --- Fine-Tuning: Calculate Metrics ---

# This cell is responsible for loading the configuration and test basin IDs for the
# selected fine-tuned model, and then either calculating or loading its performance metrics.
# It mirrors the process used for the base model but focuses on the fine-tuned experiment.

# Load configuration and basin IDs for the fine-tuned model.
# `backend.load_model_config_and_basins` reads the configuration file associated with
# the `finetune_run_dir` (which was set in the cell above).
# It extracts the experiment's configuration details, as well as the training and testing
# basin IDs used for this specific fine-tuned run.
finetune_config, finetune_train_basin_ids, finetune_test_basin_ids = backend.load_model_config_and_basins(finetune_run_dir)

# Load simulation data and calculate/load metrics for the fine-tuned model.
# `backend.load_data_and_metrics` performs the heavy lifting of reading raw model outputs
# and, based on the `CALCULATE_STATISTICS` flag, computes or loads performance metrics.
# - `model_run_dir`: The directory of the fine-tuned model run selected in the previous step.
# - `finetune_test_basin_ids`: The set of basin IDs that were used for testing this fine-tuned model.
# - `CALCULATE_STATISTICS`: A global boolean flag. If `True`, metrics are recalculated;
#   if `False`, pre-saved metrics are loaded to save computation time.
# - `model_name`: A descriptive name for the fine-tuned model, used for display purposes.
finetune_data, finetune_metrics = backend.load_data_and_metrics(
    model_run_dir=finetune_run_dir,
    test_basin_ids=finetune_test_basin_ids,
    calculate_statistics=CALCULATE_STATISTICS,
    model_name=model_name
)

# Display the head of the calculated/loaded fine-tuned model metrics DataFrame.
# This provides a quick overview of the metrics, including basin IDs, lead times, and scores.
display(finetune_metrics.head())

### Visualize Metircs

In [ ]:
# --- Fine-Tuning: Visualize Metrics ---

# This cell provides an interactive visualization to compare the performance
# of the base model against the fine-tuned model for a specific basin and metric.
# This allows you to assess the impact of fine-tuning on the target basin's predictions
# across different lead times.

# Get the list of all unique basin IDs present in the base model metrics.
# This ensures that the dropdown for basin selection contains only valid basin IDs
# for which data is available.
all_basin_ids = sorted(list(base_model_metrics.index.get_level_values('basin_id').unique()))

# Get the list of available metrics from the base model metrics columns.
# This ensures the dropdown only presents metrics that have actually been calculated and are present.
available_metrics = list(base_model_metrics.columns)

# Create a dropdown widget for selecting the basin.
# The default value is set to the `FINE_TUNING_BASIN`.
# We ensure it matches the format in all_basin_ids.
basin_widget = widgets.Dropdown(
    options=all_basin_ids,
    value=FINE_TUNING_BASIN if FINE_TUNING_BASIN in all_basin_ids else all_basin_ids[0],
    description='Select Basin:',
    disabled=False,
    style = {'description_width': 'initial'}
)

# Create a dropdown widget for selecting the metric.
# The default metric is 'KGE' (Kling-Gupta Efficiency), a common hydrological performance metric.
metric_widget = widgets.Dropdown(
    options=available_metrics,
    value='KGE',
    description='Select Metric:',
    disabled=False,
    style = {'description_width': 'initial'}
)

# Create an interactive widget linking the dropdowns to the plotting function.
# `plot_comparison_metrics_vs_lead_time` will dynamically update the plot
# whenever a new basin or metric is selected from the dropdowns.
# `widgets.fixed` is used to pass the metrics DataFrames as static arguments
# because they do not change with dropdown selections.
interactive_plot = interactive(
    backend.plot_comparison_metrics_vs_lead_time,
    base_metrics_df=widgets.fixed(base_model_metrics), # Pass the base metrics DataFrame
    finetune_metrics_df=widgets.fixed(finetune_metrics),
    basin_id=basin_widget,
    metric_name=metric_widget
)

# Display the interactive widget and the plot output.
# The `interactive_plot` object contains the controls (dropdowns) and the output (plot).
# We separate them to display controls above the plot.
ui = interactive_plot.children[:-1] # Controls are all children except the last one (the output)
out = interactive_plot.children[-1] # The output is the last child

display(VBox(ui), out) # Display controls and output in a VBox

### Visualize Hydrographs

In [ ]:
# --- Fine-Tuning: Visualize Hydrographs ---

# This cell generates a hydrograph plot to visually compare the observed streamflow
# against the simulated streamflow from both the base model and the fine-tuned model.
# This visualization is critical for understanding the models' ability to reproduce
# the hydrological behavior of the selected basin, especially at lead time zero (the current day's prediction).

# Define the lead time for the hydrograph comparison. LEAD_TIME_ZERO typically refers
# to the hindcast or current-day prediction, which is often the most accurate.
LEAD_TIME_ZERO = 0

# Normalize the fine-tuning basin ID for consistent data selection.
# The `basin_widget.value` holds the currently selected basin from the interactive dropdown
# in the previous cell (Ii3MpfSvCZ9U), ensuring the hydrograph is plotted for the user's chosen basin.
finetune_basin = basin_widget.value

# Select observed streamflow data for the chosen fine-tuning basin at the specified lead time.
# `finetune_data` contains the loaded observed data (streamflow_obs) for the fine-tuned run.
obs_hydrograph = finetune_data['streamflow_obs'].sel(
    basin=finetune_basin,
    time_step=LEAD_TIME_ZERO
)

# Select base model simulated streamflow data for the chosen fine-tuning basin at lead time 0.
# `base_model_data` contains the loaded simulation results (streamflow_sim) for the base model.
base_model_hydrograph = base_model_data['streamflow_sim'].sel(
    basin=finetune_basin,
    time_step=LEAD_TIME_ZERO
)

# Select fine-tuned model simulated streamflow data for the chosen fine-tuning basin at lead time 0.
# `finetune_data` also contains the simulation results for the fine-tuned model.
finetune_model_hydrograph = finetune_data['streamflow_sim'].sel(
    basin=finetune_basin,
    time_step=LEAD_TIME_ZERO
)

# Create the plot figure.
plt.figure(figsize=(12, 6))

# Plot the hydrographs using different colors and line styles for easy differentiation.
# Observed streamflow is shown in black, base model in blue (dashed), and fine-tuned model in red (dash-dot).
obs_hydrograph.plot(label='Observed', color='black')
base_model_hydrograph.plot(label='Base Model', color='blue', linestyle='--')
finetune_model_hydrograph.plot(label='Fine-Tuned Model', color='red', linestyle='-.')

# Add title, axis labels, and a grid for clarity.
plt.title(f"Hydrograph Comparison for Basin {FINE_TUNING_BASIN} (Lead Time {LEAD_TIME_ZERO})")
plt.xlabel("Date")
plt.ylabel("Streamflow")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend() # Display the legend to identify each line
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show() # Display the generated plot

## Look at Basin Area Distribution

In [ ]:
# Load attributes data from the specified CSV file.
# This DataFrame contains various hydrological attributes for different basins, including their area.
attributes_df = pd.read_csv(ATTRIBUTES_FILE_PATH)

# Get the fine-tuning basin ID from the `basin_widget` (from the previous interactive cell).
# This ensures that the highlighted basin in the plot is the one currently selected by the user.
finetune_basin_id = basin_widget.value

# Filter the `attributes_df` to include only the basins that were used for training.
# We select both the original 'gauge_id' and the 'area' for these basins.
training_basin_areas = attributes_df[
    attributes_df['gauge_id'].isin(train_basin_ids)
][['gauge_id', 'area']]

# Get the area of the fine-tuning basin from `attributes_df`.
# We check if `attributes_df` and its required columns exist to prevent errors.
# If the fine-tuning basin's area is found, it's stored; otherwise, a warning is printed.
if 'attributes_df' in globals() and 'gauge_id' in attributes_df.columns and 'area' in attributes_df.columns:
    finetune_basin_area_row = attributes_df[attributes_df['gauge_id'] == finetune_basin_id]
    if not finetune_basin_area_row.empty:
        finetune_basin_area = finetune_basin_area_row['area'].iloc[0]
    else:
        finetune_basin_area = None
        print(f"Warning: Area not found for fine-tuning basin ID: {finetune_basin_id}")
else:
    finetune_basin_area = None
    print("Warning: attributes_df not available or missing required columns.")


# Plot a histogram of the training basin areas.
# This visualizes the distribution of catchment areas for the basins the base model was trained on.
plt.figure(figsize=(10, 6))
plt.hist(training_basin_areas['area'], bins=50, alpha=0.7, label='Training Basin Areas')

# Add a vertical line or marker for the fine-tuning basin area if it was found.
# This helps to visually compare the size of the fine-tuning basin relative to the training basins.
if finetune_basin_area is not None:
    plt.axvline(finetune_basin_area, color='red', linestyle='dashed', linewidth=2, label=f'Fine-Tuning Basin Area ({finetune_basin_area:.2f})')

# Set plot labels and title for clarity.
plt.xlabel("Catchment Area")
plt.ylabel("Frequency")
plt.title("Distribution of Training Basin Areas with Fine-Tuning Basin Area Highlighted")
plt.legend() # Display the legend to differentiate between training areas and the fine-tuning basin
plt.grid(axis='y', linestyle='--', alpha=0.7) # Add a horizontal grid for readability
plt.show() # Display the generated plot

### Why is Basin Area Distribution Important for Fine-Tuning?

The basin area distribution plot (from the cell above) is crucial for understanding our fine-tuning strategy, especially concerning the `static_attributes_fc` layer. Here's why:

When we fine-tune a pre-trained model, our goal is often to improve its performance on a specific, potentially 'out-of-distribution' target. The base model was trained on a set of basins, and its performance might be suboptimal for basins with characteristics significantly different from those it was trained on.

1.  **Basin Area as a Key Characteristic:** Basin area is a fundamental static attribute of a catchment. If the fine-tuning basin's area falls significantly outside the range or distribution of the training basin areas (as visualized in the histogram), it suggests that the base model might not have learned a robust representation for basins of that particular size.

2.  **Role of `static_attributes_fc` Layer:** The `static_attributes_fc` (static attributes fully connected) layer in the model is specifically designed to process these static basin characteristics (like area, elevation, geology, etc.). It learns to embed these attributes into a representation that the rest of the hydrological model can use.

3.  **Why Fine-Tune This Layer:** If our target fine-tuning basin has static attributes (like area) that are dissimilar to the bulk of the training data, the `static_attributes_fc` layer might be producing a suboptimal embedding for it. By fine-tuning *only* this layer, we allow the model to learn an improved, more accurate representation for the unique static characteristics of our target basin, without retraining the entire, more complex hydrological prediction part of the model. This targeted approach is more efficient and prevents 'catastrophic forgetting'—where retraining the whole model might degrade performance on the attributes it already learned well for the original training distribution.

In essence, if the target basin is an outlier in terms of area, fine-tuning `static_attributes_fc` directly addresses this discrepancy by teaching the model how to better interpret and utilize that specific characteristic for the target basin's predictions.